<a href="https://colab.research.google.com/github/gogzicole/Hamoye-Data-science/blob/master/Data_munging_and_Web_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import json

**loading data as pandas dataframe**

In [72]:
df = pd.read_html('https://en.wikipedia.org/w/index.php?title=Fortune_Global_500&oldid=855890446', header=0)[0]
df

,Rank,Company,Country,Industry,Revenue in USD
0,1,Walmart,United States,Retail,$500 billion
1,2,State Grid,China,Power,$349 billion
2,3,Sinopec Group,China,Petroleum,$327 billion
3,4,China National Petroleum,China,Petroleum,$326 billion
4,5,Royal Dutch Shell,Netherlands,Petroleum,$312 billion
5,6,Toyota Motor,Japan,Automobiles,$265 billion
6,7,Volkswagen,Germany,Automobiles,$260 billion
7,8,BP,United Kingdom,Petroleum,$245 billion
8,9,Exxon Mobil,United States,Petroleum,$244 billion
9,10,Berkshire Hathaway,United States,Products,$242 billion


**loading the data as a json file**

In [3]:
fortune_500 = json.loads(df.to_json(orient="records"))
fortune_500[:2]

[{'Company': 'Walmart',
  'Country': 'United States',
  'Industry': 'Retail',
  'Rank': 1,
  'Revenue in USD': '$500 billion'},
 {'Company': 'State Grid',
  'Country': 'China',
  'Industry': 'Power',
  'Rank': 2,
  'Revenue in USD': '$349 billion'}]

**manually creating data with similar field as previous data**

In [8]:
other_data = [
    {"name": "Walmart",
     "employees": 2300000,
     "year founded": 1962
    },
    {"name": "State Grid Corporation of China",
     "employees": 927839,
     "year founded": 2002},
    {"name": "China Petrochemical Corporation",
     "employees":358571,
     "year founded": 1998
     },
    {"name": "China National Petroleum Corporation",
     "employees": 1636532,
     "year founded": 1988},
    {"name": "Toyota Motor Corporation",
     "employees": 364445,
     "year founded": 1937},
    {"name": "Volkswagen AG",
     "employees": 642292,
     "year founded": 1937},
    {"name": "Royal Dutch Shell",
     "employees": 92000,
     "year founded": 1907},
    {"name": "Berkshire Hathaway Inc.",
     "employees":377000,
     "year founded": 1839},
    {"name": "Apple Inc.",
     "employees": 123000,
     "year founded": 1976},
    {"name": "Exxon Mobile Corporation",
     "employees": 69600,
     "year founded": 1999},
    {"name": "BP plc",
     "employees": 74000,
     "year founded": 1908}
]

**converting other_data to a dictionary of dictionary**

In [9]:
dict_data = {i['name']: i for i in other_data }

**define a function that reads the first 5 fields of a dictionary of dictionaries**

In [10]:
def dict_head(dict_):
     return dict(list(dict_.items())[:5])

In [7]:
dict_head(dict_data)

{'China National Petroleum Corporation': {'employees': 1636532,
  'name': 'China National Petroleum Corporation',
  'year founded': 1988},
 'China Petrochemical Corporation': {'employees': 358571,
  'name': 'China Petrochemical Corporation',
  'year founded': 1998},
 'State Grid Corporation of China': {'employees': 927839,
  'name': 'State Grid Corporation of China',
  'year founded': 2002},
 'Toyota Motor Corporation': {'employees': 364445,
  'name': 'Toyota Motor Corporation',
  'year founded': 1937},
 'Walmart': {'employees': 2300000, 'name': 'Walmart', 'year founded': 1962}}

**creates a mapping variable that contains a key value pair of names from df and other_data**

In [23]:
mapping = {
    'Apple': 'Apple Inc.',
    'BP': 'BP plc',
    'Berkshire Hathaway': 'Berkshire Hathaway Inc.',
    'China National Petroleum': 'China National Petroleum Corporation',
    'Exxon Mobil': 'Exxon Mobile Corporation',
    'Sinopec Group': 'China Petrochemical Corporation',
    'State Grid': 'State Grid Corporation of China',
    'Toyota Motor': 'Toyota Motor Corporation',
    'Volkswagen': 'Volkswagen AG'
}

In [49]:
name = 'Exxon Mobil'
dict_data[mapping.get(name,name)].get('employees')

69600

**defining a function that strips alphabets from numerals and coverts to float**

In [26]:
def convert_revenue(x):
    return float(x.lstrip('$').rstrip('billion')) * 1e9

assert convert_revenue('$500 billion') == 500e9

In [65]:
[convert_revenue(i['Revenue in USD']) for i in fortune_500]

[500000000000.0,
 349000000000.0,
 327000000000.0,
 326000000000.0,
 312000000000.0,
 265000000000.0,
 260000000000.0,
 245000000000.0,
 244000000000.0,
 242000000000.0]

**define a function that computes rev_per_employees and add new field to data set**

In [27]:
def rev_per_emp(company):
    name = company[u'Company']
    n_employees = dict_data[mapping.get(name, name)].get('employees')
    company['rev per emp'] = convert_revenue(company[u'Revenue in USD'])/n_employees
    return company

def compute_copy(d, func):
    return func({k:v for k,v in d.items()})

data = list(map(lambda x : compute_copy(x, rev_per_emp), fortune_500))

In [73]:
data[:2]

[{'Company': 'Walmart',
  'Country': 'United States',
  'Industry': 'Retail',
  'Rank': 1,
  'Revenue in USD': '$500 billion',
  'rev per emp': 217391.30434782608},
 {'Company': 'State Grid',
  'Country': 'China',
  'Industry': 'Power',
  'Rank': 2,
  'Revenue in USD': '$349 billion',
  'rev per emp': 376142.84374767606}]

**creates a sorted list of revenue per employees**

In [74]:
rev_per_emp = sorted([(i[u'Company'], i['rev per emp']) for i in data], 
                   key=lambda x : x[1],
                   reverse=True)
rev_per_emp

[('Exxon Mobil', 3505747.1264367816),
 ('Royal Dutch Shell', 3391304.347826087),
 ('BP', 3310810.810810811),
 ('Sinopec Group', 911953.2812190612),
 ('Toyota Motor', 727133.0379069544),
 ('Berkshire Hathaway', 641909.8143236075),
 ('Volkswagen', 404800.30889377417),
 ('State Grid', 376142.84374767606),
 ('Walmart', 217391.30434782608),
 ('China National Petroleum', 199201.72657790987)]

**counts the number of unique industries**

In [75]:
from collections import Counter
Counter(i[u'Industry'] for i in data)

Counter({'Automobiles': 2,
         'Petroleum': 5,
         'Power': 1,
         'Products': 1,
         'Retail': 1})

In [77]:
sub_data = [i for i in data if i[u'Industry'] in [u'Automobiles', u'Petroleum']]
sub_data[:2]

[{'Company': 'Sinopec Group',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Rank': 3,
  'Revenue in USD': '$327 billion',
  'rev per emp': 911953.2812190612},
 {'Company': 'China National Petroleum',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Rank': 4,
  'Revenue in USD': '$326 billion',
  'rev per emp': 199201.72657790987}]

**Loading new datasets **

In [73]:
df_list = pd.read_html("https://en.wikipedia.org/w/index.php?title=Automotive_industry&oldid=875776152", header=0)
car_totals = json.loads(df_list[0].to_json(orient="records"))
car_by_man = json.loads(df_list[2].to_json(orient='records'))

In [74]:
car_totals[:2]

[{'Change': '—', 'Production': 54434000, 'Source': '[17]', 'Year': 1997},
 {'Change': '2.7%', 'Production': 52987000, 'Source': '[17]', 'Year': 1998}]

In [75]:
car_by_man[:2]

[{'Country': 'Japan', 'Group': 'Toyota', 'Rank': 1, 'Vehicles': 10213486},
 {'Country': 'Germany',
  'Group': 'Volkswagen Group',
  'Rank': 2,
  'Vehicles': 10126281}]

In [76]:
total_prod = sorted((i[u"Year"], i[u'Production']) for i in car_totals)[-1][1]
total_prod

97302534

In [77]:
car_by_man_dict = {i[u'Group']:i[u'Vehicles'] for i in car_by_man}
market_share = {}
for name, orig_name in zip(['Toyota', 'Volkswagen Group'], ['Toyota', 'Volkswagen']):
    market_share[orig_name] = car_by_man_dict[name]/ float(total_prod)
    
market_share

{'Toyota': 0.1049662899837737, 'Volkswagen': 0.10407006460900597}

In [78]:
rev = pd.read_html("https://en.wikipedia.org/w/index.php?title=List_of_largest_oil_and_gas_companies_by_revenue&oldid=871711850", header=1)[0]
rev = rev.iloc[:, 1:3]
rev.columns = ['Company', 'Revenue']
rev = rev[~(rev['Company'] == 'Company name')]
oil_data = json.loads(rev.to_json(orient="records"))
oil_data[:2]

[{'Company': 'Saudi Aramco', 'Revenue': '465.49'},
 {'Company': 'Sinopec Group', 'Revenue': '448.00'}]

In [93]:
total = sum([float(i[u'Revenue'].rstrip('*')) for i in oil_data])
shares = {i[u'Company']:float(i[u'Revenue'].rstrip('*'))/total for i in oil_data}
print(total)

5482.090000000001


In [95]:
petro_companies = [i[u'Company'] for i in data if i['Industry'] == u'Petroleum']
petro_companies

['Sinopec Group',
 'China National Petroleum',
 'Royal Dutch Shell',
 'BP',
 'Exxon Mobil']

In [96]:
[(i, i in shares) for i in petro_companies]

[('Sinopec Group', True),
 ('China National Petroleum', False),
 ('Royal Dutch Shell', True),
 ('BP', True),
 ('Exxon Mobil', True)]

In [97]:
shares.keys()

dict_keys(['Saudi Aramco', 'Sinopec Group', 'China National Petroleum Corporation', 'Exxon Mobil', 'Royal Dutch Shell', 'Kuwait Petroleum Corporation', 'BP', 'Total SA', 'Lukoil', 'Eni', 'Valero Energy', 'Chevron Corporation', 'PDVSA', 'Pemex', 'National Iranian Oil Company', 'Gazprom', 'Phillips 66', 'Petronas', 'China National Offshore Oil', 'Marathon Petroleum', 'PTT', 'Rosneft', 'JX Holdings', 'Engie', 'Petrobras', 'Equinor', 'Indian Oil Corporation', 'Sonatrach', 'Reliance Industries', 'Pertamina', 'SOCAR', 'GS Caltex', 'Enterprise Products', 'Repsol', 'Centrica', 'Bharat Petroleum', 'OMV Group', 'Idemitsu Kosan', 'Hindustan Petroleum', 'PKN Orlen', 'Schlumberger', 'Suncor Energy', 'Hellenic Petroleum', 'ConocoPhillips', 'Motor Oil Hellas', 'Ecopetrol', 'CEPSA', 'MOL'])

In [68]:
def fuzzy_match(word, s):
    words = set(word.split(' '))
    overlaps = [(k, len(v.intersection(words))) for k, v in s.items()]
    return max(overlaps, key=lambda x : x[1])[0]

In [99]:
split_names = {i: set(i.split(' ')) for i in shares.keys()}
for i in petro_companies:
    match = fuzzy_match(i, split_names)
    print("matched {} to {}".format(i, match))
    market_share[i] = shares[match]

matched Sinopec Group to Sinopec Group
matched China National Petroleum to China National Petroleum Corporation
matched Royal Dutch Shell to Royal Dutch Shell
matched BP to BP
matched Exxon Mobil to Exxon Mobil


In [100]:
market_share

{'BP': 0.040641434197541446,
 'China National Petroleum': 0.07818550954106918,
 'Exxon Mobil': 0.0490506357976611,
 'Royal Dutch Shell': 0.04833922828702191,
 'Sinopec Group': 0.0817206576323993,
 'Toyota': 0.1049662899837737,
 'Volkswagen': 0.10407006460900597}

**Using Pandas**

In [11]:
df

,Rank,Company,Country,Industry,Revenue in USD
0,1,Walmart,United States,Retail,$500 billion
1,2,State Grid,China,Power,$349 billion
2,3,Sinopec Group,China,Petroleum,$327 billion
3,4,China National Petroleum,China,Petroleum,$326 billion
4,5,Royal Dutch Shell,Netherlands,Petroleum,$312 billion
5,6,Toyota Motor,Japan,Automobiles,$265 billion
6,7,Volkswagen,Germany,Automobiles,$260 billion
7,8,BP,United Kingdom,Petroleum,$245 billion
8,9,Exxon Mobil,United States,Petroleum,$244 billion
9,10,Berkshire Hathaway,United States,Products,$242 billion


In [54]:
df['rev'] = (df['Revenue in USD']).apply(convert_revenue)
df['Employees'] = (df['Company']).apply(lambda x: dict_data[mapping.get(x,x)].get('employees'))
df['rev per empl'] = df['rev']/df['Employees'].astype(float)
df.sort_values(by = 'rev per empl', ascending = False)

,Rank,Company,Country,Industry,Revenue in USD,rev,Employees,rev per empl
8,9,Exxon Mobil,United States,Petroleum,$244 billion,2.440000e+11,69600,3.505747e+06
4,5,Royal Dutch Shell,Netherlands,Petroleum,$312 billion,3.120000e+11,92000,3.391304e+06
7,8,BP,United Kingdom,Petroleum,$245 billion,2.450000e+11,74000,3.310811e+06
2,3,Sinopec Group,China,Petroleum,$327 billion,3.270000e+11,358571,9.119533e+05
5,6,Toyota Motor,Japan,Automobiles,$265 billion,2.650000e+11,364445,7.271330e+05
9,10,Berkshire Hathaway,United States,Products,$242 billion,2.420000e+11,377000,6.419098e+05
6,7,Volkswagen,Germany,Automobiles,$260 billion,2.600000e+11,642292,4.048003e+05
1,2,State Grid,China,Power,$349 billion,3.490000e+11,927839,3.761428e+05
0,1,Walmart,United States,Retail,$500 billion,5.000000e+11,2300000,2.173913e+05
3,4,China National Petroleum,China,Petroleum,$326 billion,3.260000e+11,1636532,1.992017e+05


**Computing the market shares**

In [56]:
df_list = pd.read_html("https://en.wikipedia.org/w/index.php?title=Automotive_industry&oldid=875776152", header=0)
df_totals = df_list[0]
df_by_man = df_list[2]

In [59]:
total_prod = df_totals.sort_values(by='Year').iloc[-1]['Production']
total_prod

97302534

In [60]:
df_by_man['share'] = df_by_man['Vehicles'].astype(float) / total_prod
market_share = df_by_man.set_index('Group')['share'][['Toyota', 'Volkswagen Group']]
market_share

Group
Toyota              0.104966
Volkswagen Group    0.104070
Name: share, dtype: float64

In [62]:
rev = pd.read_html("https://en.wikipedia.org/w/index.php?title=List_of_largest_oil_and_gas_companies_by_revenue&oldid=871711850", header=1)[0]
rev = rev.iloc[:, 1:3]
rev.columns = ['Company', 'Revenue']
rev = rev[~(rev['Company'] == 'Company name')]
rev.head()

,Company,Revenue
0,Saudi Aramco,465.49
1,Sinopec Group,448.00
2,China National Petroleum Corporation,428.62
3,Exxon Mobil,268.9
4,Royal Dutch Shell,265


In [63]:
rev['rev_clean'] = rev['Revenue'].apply(lambda x : float(x.rstrip('*')))
total = rev['rev_clean'].sum()
total

5482.09

In [65]:
rev['share'] = rev['rev_clean'] / total
rev.head()

,Company,Revenue,rev_clean,share
0,Saudi Aramco,465.49,465.49,0.084911
1,Sinopec Group,448.00,448.00,0.081721
2,China National Petroleum Corporation,428.62,428.62,0.078186
3,Exxon Mobil,268.9,268.90,0.049051
4,Royal Dutch Shell,265,265.00,0.048339


In [66]:
rev = rev[rev['Company'].isin(['Exxon Mobil', 'Sinopec', 'China National Petroleum Corporation', 'Royal Dutch Shell'])].copy()
rev

,Company,Revenue,rev_clean,share
2,China National Petroleum Corporation,428.62,428.62,0.078186
3,Exxon Mobil,268.9,268.90,0.049051
4,Royal Dutch Shell,265,265.00,0.048339


In [69]:
# do fuzzy search
split_names = {i: set(i.split(' ')) for i in df['Company']}

def fuzzy(word):
    return fuzzy_match(word, split_names)

rev['name'] = rev['Company'].apply(fuzzy)
rev

,Company,Revenue,rev_clean,share,name
2,China National Petroleum Corporation,428.62,428.62,0.078186,China National Petroleum
3,Exxon Mobil,268.9,268.90,0.049051,Exxon Mobil
4,Royal Dutch Shell,265,265.00,0.048339,Royal Dutch Shell


In [70]:
ms2 = df.merge(rev[['share', 'name']], left_on='Company', right_on='name')

**Now we want to put these together and get only the company and the market share.**

In [71]:
ms = market_share.reset_index()[['Group','share']]
ms.columns = ['Company', 'share']
pd.concat([ms, ms2[['Company', 'share']]])

,Company,share
0,Toyota,0.104966
1,Volkswagen Group,0.104070
0,China National Petroleum,0.078186
1,Royal Dutch Shell,0.048339
2,Exxon Mobil,0.049051
